In [12]:
import pandas as pd
import numpy as np
import os
import time
import random
import pprint
import yaml
import warnings
warnings.filterwarnings(action="ignore")


import pandas as pd
import numpy as np
import os
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder

from sklearn.model_selection import KFold
import random
toacco_geo = pd.read_csv("feature/toacco_geo2_mit_order.csv")
toacco_geo_chem = pd.read_csv("feature/toacco_geo_chem_erase_mit_order.csv")
toacco_geo = toacco_geo.iloc[:,2:]
toacco_geo.head()

,name,topology,First nodular symmetry code,First nodular character,First nodular ID,Second nodular symmetry code,Second nodular character,Second nodular ID,Connecting building block ID,Di,...,VPOV,GPOV,POAV_vol_frac,PONAV_vol_frac,GPOAV,GPONAV,POAV,PONAV,bulk,shear
0,acs_sym_7_mc_4_L_12,acs,7,metaliic,4,7,metallic,4,12,9.64336,...,0.6635,0.946242,0.6635,0.0,0.946242,0.0,2087.70,0.0,14.128650,11.231780
1,acs_sym_7_mc_4_L_13,acs,7,metaliic,4,7,metallic,4,13,9.94957,...,0.7011,0.953612,0.7011,0.0,0.953611,0.0,2141.51,0.0,14.759740,11.627400
2,acs_sym_7_mc_4_L_20,acs,7,metaliic,4,7,metallic,4,20,13.42609,...,0.7960,1.846966,0.7960,0.0,1.846970,0.0,4517.02,0.0,8.911607,5.467771
3,acs_sym_7_mc_4_L_24,acs,7,metaliic,4,7,metallic,4,24,15.06601,...,0.8043,2.149081,0.8043,0.0,2.149080,0.0,6370.94,0.0,6.867950,4.384087
4,acs_sym_7_mc_4_L_26,acs,7,metaliic,4,7,metallic,4,26,15.35307,...,0.8313,2.061613,0.8313,0.0,2.061610,0.0,6273.98,0.0,7.366024,5.053149


In [3]:
geometric = ['Di', 'Df', 'Dif', 'rho', 'VSA','GSA', 'VPOV', 'GPOV', 'POAV_vol_frac', 'PONAV_vol_frac', 'GPOAV','GPONAV', 'POAV', 'PONAV']
RAC = ['f-chi-0-all', 'f-chi-1-all', 'f-chi-2-all',
       'f-chi-3-all', 'f-Z-0-all', 'f-Z-1-all', 'f-Z-2-all', 'f-Z-3-all',
       'f-I-0-all', 'f-I-1-all', 'f-I-2-all', 'f-I-3-all', 'f-T-0-all',
       'f-T-1-all', 'f-T-2-all', 'f-T-3-all', 'f-S-0-all', 'f-S-1-all',
       'f-S-2-all', 'f-S-3-all', 'mc-chi-0-all', 'mc-chi-1-all',
       'mc-chi-2-all', 'mc-chi-3-all', 'mc-Z-0-all', 'mc-Z-1-all',
       'mc-Z-2-all', 'mc-Z-3-all', 'mc-I-0-all', 'mc-I-1-all', 'mc-I-2-all',
       'mc-I-3-all', 'mc-T-0-all', 'mc-T-1-all', 'mc-T-2-all', 'mc-T-3-all',
       'mc-S-0-all', 'mc-S-1-all', 'mc-S-2-all', 'mc-S-3-all',
       'D_mc-chi-0-all', 'D_mc-chi-1-all', 'D_mc-chi-2-all', 'D_mc-chi-3-all',
       'D_mc-Z-0-all', 'D_mc-Z-1-all', 'D_mc-Z-2-all', 'D_mc-Z-3-all',
       'D_mc-I-0-all', 'D_mc-I-1-all', 'D_mc-I-2-all', 'D_mc-I-3-all',
       'D_mc-T-0-all', 'D_mc-T-1-all', 'D_mc-T-2-all', 'D_mc-T-3-all',
       'D_mc-S-0-all', 'D_mc-S-1-all', 'D_mc-S-2-all', 'D_mc-S-3-all',
       'f-lig-chi-0', 'f-lig-chi-1', 'f-lig-chi-2', 'f-lig-chi-3', 'f-lig-Z-0',
       'f-lig-Z-1', 'f-lig-Z-2', 'f-lig-Z-3', 'f-lig-I-0', 'f-lig-I-1',
       'f-lig-I-2', 'f-lig-I-3', 'f-lig-T-0', 'f-lig-T-1', 'f-lig-T-2',
       'f-lig-T-3', 'f-lig-S-0', 'f-lig-S-1', 'f-lig-S-2', 'f-lig-S-3',
       'lc-chi-0-all', 'lc-chi-1-all', 'lc-chi-2-all', 'lc-chi-3-all',
       'lc-Z-0-all', 'lc-Z-1-all', 'lc-Z-2-all', 'lc-Z-3-all', 'lc-I-0-all',
       'lc-I-1-all', 'lc-I-2-all', 'lc-I-3-all', 'lc-T-0-all', 'lc-T-1-all',
       'lc-T-2-all', 'lc-T-3-all', 'lc-S-0-all', 'lc-S-1-all', 'lc-S-2-all',
        'lc-S-3-all', 'lc-alpha-0-all', 'lc-alpha-1-all', 'lc-alpha-2-all',
       'lc-alpha-3-all', 'D_lc-chi-0-all', 'D_lc-chi-1-all', 'D_lc-chi-2-all',
       'D_lc-chi-3-all', 'D_lc-Z-0-all', 'D_lc-Z-1-all', 'D_lc-Z-2-all',
       'D_lc-Z-3-all', 'D_lc-I-0-all', 'D_lc-I-1-all', 'D_lc-I-2-all',
       'D_lc-I-3-all', 'D_lc-T-0-all', 'D_lc-T-1-all', 'D_lc-T-2-all',
       'D_lc-T-3-all', 'D_lc-S-0-all', 'D_lc-S-1-all', 'D_lc-S-2-all',
       'D_lc-S-3-all', 'D_lc-alpha-0-all', 'D_lc-alpha-1-all',
       'D_lc-alpha-2-all', 'D_lc-alpha-3-all', 'func-chi-0-all',
       'func-chi-1-all', 'func-chi-2-all', 'func-chi-3-all', 'func-Z-0-all',
       'func-Z-1-all', 'func-Z-2-all', 'func-Z-3-all', 'func-I-0-all',
       'func-I-1-all', 'func-I-2-all', 'func-I-3-all', 'func-T-0-all',
       'func-T-1-all', 'func-T-2-all', 'func-T-3-all', 'func-S-0-all',
       'func-S-1-all', 'func-S-2-all', 'func-S-3-all', 'func-alpha-0-all',
       'func-alpha-1-all', 'func-alpha-2-all', 'func-alpha-3-all',
       'D_func-chi-0-all', 'D_func-chi-1-all', 'D_func-chi-2-all',
       'D_func-chi-3-all', 'D_func-Z-0-all', 'D_func-Z-1-all',
       'D_func-Z-2-all', 'D_func-Z-3-all', 'D_func-I-0-all', 'D_func-I-1-all',
       'D_func-I-2-all', 'D_func-I-3-all', 'D_func-T-0-all', 'D_func-T-1-all',
       'D_func-T-2-all', 'D_func-T-3-all', 'D_func-S-0-all', 'D_func-S-1-all',
       'D_func-S-2-all', 'D_func-S-3-all', 'D_func-alpha-0-all',
       'D_func-alpha-1-all', 'D_func-alpha-2-all', 'D_func-alpha-3-all']
property_ = ["bulk"]
## define the domain
geo = geometric + property_
RAC_geo = geometric + RAC + property_

In [4]:
X = np.array(toacco_geo_chem[geometric])
y = np.array(toacco_geo_chem[property_])

In [5]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate

kf = KFold(n_splits = 10, shuffle = True, random_state = 42)

loss_history = []
accuracy = []

for train_index, test_index in kf.split(X):

    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    model = RandomForestRegressor(random_state=42) 
    model.fit(X_train, y_train) 

    y_pred = model.predict(X_test) 
    loss_history.append(mean_squared_error(y_test, y_pred,squared=False))
    accuracy.append(r2_score(y_test,y_pred,))
print("history of rmse :", loss_history)
print("average rmse value  :", np.mean(loss_history))
print("history of r2 square :",accuracy)
print("averare r2 square :" ,np.mean(accuracy))

history of rmse : [3.191416893308912, 2.359654586028724, 3.0162416990631638, 2.827681075030097, 3.429882725706608, 3.6592499817517763, 3.2122427329222534, 2.9233594038675283, 2.960978568090716, 2.295239741338949]
average rmse value  : 2.9875947407108727
history of r2 square : [0.783879500457479, 0.8354448289959093, 0.8136957926689059, 0.8001608747667639, 0.8348120672117586, 0.7924127333617775, 0.6492043101226055, 0.8235160816029098, 0.7678086691896204, 0.850456684855854]
averare r2 square : 0.7951391543233585


In [6]:
import optuna
from sklearn import datasets

def objective(trial,X=X, y=y):
    
        
    n_estimators = trial.suggest_int("n_estimators", 50, 500, 50)
        
    max_features = trial.suggest_categorical("max_features",["auto","sqrt","log2"])
        
    max_depth = trial.suggest_categorical("max_depth",[None,10,20,30,40,50,60,70,80,90,100])
        
    min_samples_split = trial.suggest_int("min_samples_split",2,10,2)
        
    min_samples_leaf = trial.suggest_int("min_samples_leaf",1,5,1)
        
    bootstrap = trial.suggest_categorical("bootstrap",[True,False])

    accuracies = []
    
    kf = KFold(n_splits = 10, shuffle = True, random_state = 42)

    loss = []

    for train_index, test_index in kf.split(X):

        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        model = RandomForestRegressor(random_state=42,
                                      n_estimators=n_estimators,
                                      max_features = max_features,
                                      max_depth = max_depth,
                                      min_samples_split = min_samples_split,
                                      min_samples_leaf = min_samples_leaf,
                                      bootstrap = bootstrap) 
        model.fit(X_train, y_train) 

        y_pred = model.predict(X_test) 
        
        rmse = mean_squared_error(y_test,y_pred,squared = False)
        
        loss.append(rmse)
        average_loss = np.mean(loss)
        
    return average_loss
    
    
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials = 160)

print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)
print("best rmse is :",study.best_trial.values)

[I 2022-11-24 16:12:41,386] A new study created in memory with name: no-name-3a8d25a7-52ce-4d40-b54d-cbfa003459b1
[I 2022-11-24 16:12:56,725] Trial 0 finished with value: 2.9539699261087327 and parameters: {'n_estimators': 350, 'max_features': 'sqrt', 'max_depth': 20, 'min_samples_split': 8, 'min_samples_leaf': 3, 'bootstrap': False}. Best is trial 0 with value: 2.9539699261087327.
[I 2022-11-24 16:13:59,148] Trial 1 finished with value: 3.6661314842497013 and parameters: {'n_estimators': 400, 'max_features': 'auto', 'max_depth': 70, 'min_samples_split': 8, 'min_samples_leaf': 5, 'bootstrap': False}. Best is trial 0 with value: 2.9539699261087327.
[I 2022-11-24 16:14:10,212] Trial 2 finished with value: 2.955556259395867 and parameters: {'n_estimators': 250, 'max_features': 'sqrt', 'max_depth': 40, 'min_samples_split': 8, 'min_samples_leaf': 3, 'bootstrap': False}. Best is trial 0 with value: 2.9539699261087327.
[I 2022-11-24 16:14:21,669] Trial 3 finished with value: 2.939211136422046

[I 2022-11-24 16:21:48,328] Trial 29 finished with value: 2.8520650704877135 and parameters: {'n_estimators': 350, 'max_features': 'sqrt', 'max_depth': 80, 'min_samples_split': 4, 'min_samples_leaf': 1, 'bootstrap': False}. Best is trial 28 with value: 2.8520650704877135.
[I 2022-11-24 16:22:03,544] Trial 30 finished with value: 2.9888417677928727 and parameters: {'n_estimators': 350, 'max_features': 'sqrt', 'max_depth': 80, 'min_samples_split': 2, 'min_samples_leaf': 4, 'bootstrap': False}. Best is trial 28 with value: 2.8520650704877135.
[I 2022-11-24 16:22:21,784] Trial 31 finished with value: 2.8520650704877135 and parameters: {'n_estimators': 350, 'max_features': 'sqrt', 'max_depth': 80, 'min_samples_split': 4, 'min_samples_leaf': 1, 'bootstrap': False}. Best is trial 28 with value: 2.8520650704877135.
[I 2022-11-24 16:22:39,957] Trial 32 finished with value: 2.8520650704877135 and parameters: {'n_estimators': 350, 'max_features': 'sqrt', 'max_depth': 80, 'min_samples_split': 4, '

[I 2022-11-24 16:29:43,364] Trial 58 finished with value: 2.9393375214379196 and parameters: {'n_estimators': 300, 'max_features': 'sqrt', 'max_depth': 70, 'min_samples_split': 2, 'min_samples_leaf': 1, 'bootstrap': True}. Best is trial 34 with value: 2.8416524921137176.
[I 2022-11-24 16:30:03,384] Trial 59 finished with value: 2.8416524921137176 and parameters: {'n_estimators': 350, 'max_features': 'sqrt', 'max_depth': 50, 'min_samples_split': 2, 'min_samples_leaf': 1, 'bootstrap': False}. Best is trial 34 with value: 2.8416524921137176.
[I 2022-11-24 16:31:00,727] Trial 60 finished with value: 3.6753547965419933 and parameters: {'n_estimators': 350, 'max_features': 'auto', 'max_depth': 50, 'min_samples_split': 2, 'min_samples_leaf': 4, 'bootstrap': False}. Best is trial 34 with value: 2.8416524921137176.
[I 2022-11-24 16:31:17,893] Trial 61 finished with value: 2.8421434948121034 and parameters: {'n_estimators': 300, 'max_features': 'sqrt', 'max_depth': 50, 'min_samples_split': 2, 'm

[I 2022-11-24 16:42:31,167] Trial 87 finished with value: 2.967941963534066 and parameters: {'n_estimators': 500, 'max_features': 'sqrt', 'max_depth': 40, 'min_samples_split': 4, 'min_samples_leaf': 1, 'bootstrap': True}. Best is trial 78 with value: 2.840297888655496.
[I 2022-11-24 16:42:59,780] Trial 88 finished with value: 2.840297888655496 and parameters: {'n_estimators': 500, 'max_features': 'sqrt', 'max_depth': 40, 'min_samples_split': 2, 'min_samples_leaf': 1, 'bootstrap': False}. Best is trial 78 with value: 2.840297888655496.
[I 2022-11-24 16:43:28,390] Trial 89 finished with value: 2.840297888655496 and parameters: {'n_estimators': 500, 'max_features': 'sqrt', 'max_depth': 40, 'min_samples_split': 2, 'min_samples_leaf': 1, 'bootstrap': False}. Best is trial 78 with value: 2.840297888655496.
[I 2022-11-24 16:43:57,072] Trial 90 finished with value: 2.840297888655496 and parameters: {'n_estimators': 500, 'max_features': 'sqrt', 'max_depth': 40, 'min_samples_split': 2, 'min_samp

[I 2022-11-24 16:56:50,893] Trial 116 finished with value: 2.9156079800452344 and parameters: {'n_estimators': 500, 'max_features': 'sqrt', 'max_depth': 40, 'min_samples_split': 8, 'min_samples_leaf': 1, 'bootstrap': False}. Best is trial 78 with value: 2.840297888655496.
[I 2022-11-24 16:57:19,914] Trial 117 finished with value: 2.840297888655496 and parameters: {'n_estimators': 500, 'max_features': 'sqrt', 'max_depth': 100, 'min_samples_split': 2, 'min_samples_leaf': 1, 'bootstrap': False}. Best is trial 78 with value: 2.840297888655496.
[I 2022-11-24 16:57:41,919] Trial 118 finished with value: 2.984398744950167 and parameters: {'n_estimators': 500, 'max_features': 'sqrt', 'max_depth': 60, 'min_samples_split': 2, 'min_samples_leaf': 4, 'bootstrap': False}. Best is trial 78 with value: 2.840297888655496.
[I 2022-11-24 16:58:07,697] Trial 119 finished with value: 2.8415099560882453 and parameters: {'n_estimators': 450, 'max_features': 'sqrt', 'max_depth': 70, 'min_samples_split': 2, '

[I 2022-11-24 17:12:28,298] Trial 145 finished with value: 2.840297888655496 and parameters: {'n_estimators': 500, 'max_features': 'sqrt', 'max_depth': 40, 'min_samples_split': 2, 'min_samples_leaf': 1, 'bootstrap': False}. Best is trial 78 with value: 2.840297888655496.
[I 2022-11-24 17:12:56,945] Trial 146 finished with value: 2.840297888655496 and parameters: {'n_estimators': 500, 'max_features': 'sqrt', 'max_depth': 40, 'min_samples_split': 2, 'min_samples_leaf': 1, 'bootstrap': False}. Best is trial 78 with value: 2.840297888655496.
[I 2022-11-24 17:13:17,318] Trial 147 finished with value: 2.9413165229012734 and parameters: {'n_estimators': 500, 'max_features': 'sqrt', 'max_depth': 40, 'min_samples_split': 2, 'min_samples_leaf': 1, 'bootstrap': True}. Best is trial 78 with value: 2.840297888655496.
[I 2022-11-24 17:13:43,196] Trial 148 finished with value: 2.8417579037402962 and parameters: {'n_estimators': 450, 'max_features': 'sqrt', 'max_depth': 30, 'min_samples_split': 2, 'mi

Number of finished trials: 160
Best trial: {'n_estimators': 500, 'max_features': 'sqrt', 'max_depth': 50, 'min_samples_split': 2, 'min_samples_leaf': 1, 'bootstrap': False}
best rmse is : [2.840297888655496]


In [7]:
params = study.best_trial.params

In [9]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate

kf = KFold(n_splits = 10, shuffle = True, random_state = 42)

loss_history = []
accuracy = []

for train_index, test_index in kf.split(X):

    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    model = RandomForestRegressor(random_state=42,**params) 
    model.fit(X_train, y_train) 

    y_pred = model.predict(X_test) 
    loss_history.append(mean_squared_error(y_test, y_pred,squared=False))
    accuracy.append(r2_score(y_test,y_pred,))
print("history of rmse :", loss_history)
print("average rmse value  :", np.mean(loss_history))
print("history of r2 square :",accuracy)
print("averare r2 square :" ,np.mean(accuracy))

history of rmse : [3.062690831017017, 2.24399512575429, 3.0035537449803478, 2.6888487052638466, 3.3129354807685396, 3.5095794841222934, 2.970353623544691, 2.518713074028338, 2.7963221768814406, 2.2959866401941555]
average rmse value  : 2.840297888655496
history of r2 square : [0.8009623661134591, 0.8511809649201438, 0.8152598897860106, 0.819302392644402, 0.8458847082882057, 0.809046898312089, 0.7000465584130491, 0.8689919123419805, 0.792914406590669, 0.8503593426025631]
averare r2 square : 0.8153949440012573


In [11]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2, random_state = 42)
model = RandomForestRegressor(random_state = 42, **params)
model.fit(X_train,y_train)

y_pred = model.predict(X_test)
print(r2_score(y_test,y_pred))
print(mean_squared_error(y_test,y_pred,squared=False))

0.8135902034855926
2.7481619166816973


# Topology 전용

In [14]:
import pandas as pd
import numpy as np
import os
import time
import random
import pprint
import yaml
import warnings
warnings.filterwarnings(action="ignore")


import pandas as pd
import numpy as np
import os
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder

from sklearn.model_selection import KFold
import random
toacco_geo = pd.read_csv("feature/toacco_geo2_mit_order.csv")
data = pd.read_csv("feature/toacco_geo_chem_erase_mit_order.csv")
data = data.iloc[:,2:]
data.head()

,MOF ID,name,topology,First nodular symmetry code,First nodular character,First nodular ID,Second nodular symmetry code,Second nodular character,Second nodular ID,Connecting building block ID,...,D_func-S-0-all,D_func-S-1-all,D_func-S-2-all,D_func-S-3-all,D_func-alpha-0-all,D_func-alpha-1-all,D_func-alpha-2-all,D_func-alpha-3-all,bulk,shear
0,3,acs_sym_7_mc_4_L_12,acs,7,metaliic,4,7,metallic,4,12,...,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,14.128650,11.231780
1,4,acs_sym_7_mc_4_L_13,acs,7,metaliic,4,7,metallic,4,13,...,0.0,-0.02,-0.04,0.02,0.0,-3.9,-7.8,0.3,14.759740,11.627400
2,12,acs_sym_7_mc_4_L_20,acs,7,metaliic,4,7,metallic,4,20,...,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,8.911607,5.467771
3,16,acs_sym_7_mc_4_L_24,acs,7,metaliic,4,7,metallic,4,24,...,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0,6.867950,4.384087
4,18,acs_sym_7_mc_4_L_26,acs,7,metaliic,4,7,metallic,4,26,...,0.0,-0.02,-0.04,0.00,0.0,-3.9,-7.8,-1.8,7.366024,5.053149


In [15]:
from sklearn.preprocessing import OneHotEncoder
geometric = ["topology",'Di', 'Df', 'Dif', 'rho', 'VSA',
       'GSA', 'VPOV', 'GPOV', 'POAV_vol_frac', 'PONAV_vol_frac', 'GPOAV',
       'GPONAV', 'POAV', 'PONAV']
X = data[geometric]
y = data["bulk"]

onehotencoder = OneHotEncoder()
top_one = onehotencoder.fit_transform(X.topology.values.reshape(-1,1)).toarray()

df_top_one = pd.DataFrame(top_one,columns=["topology_"+str(int(i)) for i in range(40)])

df = pd.concat([X, df_top_one,y], axis=1)
df= df.drop(['topology'], axis=1) 
print(df.head())

         Di        Df       Dif      rho      VSA      GSA    VPOV      GPOV  \
0   9.64336   8.80746   9.64336  3146.50  2193.80  3128.66  0.6635  0.946242   
1   9.94957   9.48351   9.94957  3054.50  2236.08  3041.44  0.7011  0.953612   
2  13.42609  11.83384  13.42609  5674.65  2069.82  4802.63  0.7960  1.846966   
3  15.06601  14.00604  15.05182  7921.10  1838.19  4911.62  0.8043  2.149081   
4  15.35307  14.52062  15.35307  7547.20  1783.35  4422.69  0.8313  2.061613   

   POAV_vol_frac  PONAV_vol_frac  ...  topology_31  topology_32  topology_33  \
0         0.6635             0.0  ...          0.0          0.0          0.0   
1         0.7011             0.0  ...          0.0          0.0          0.0   
2         0.7960             0.0  ...          0.0          0.0          0.0   
3         0.8043             0.0  ...          0.0          0.0          0.0   
4         0.8313             0.0  ...          0.0          0.0          0.0   

   topology_34  topology_35  topology_

In [16]:
X = np.array(df.iloc[:,:-1])
y = np.array(df.iloc[:,-1])
# topology + geometric one hot encoding

In [17]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate

kf = KFold(n_splits = 10, shuffle = True, random_state = 42)

loss_history = []
accuracy = []

for train_index, test_index in kf.split(X):

    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    model = RandomForestRegressor(random_state=42) 
    model.fit(X_train, y_train) 

    y_pred = model.predict(X_test) 
    loss_history.append(mean_squared_error(y_test, y_pred,squared=False))
    accuracy.append(r2_score(y_test,y_pred,))
print("history of rmse :", loss_history)
print("average rmse value  :", np.mean(loss_history))
print("history of r2 square :",accuracy)
print("averare r2 square :" ,np.mean(accuracy))

history of rmse : [2.8039619422796638, 2.155899964699111, 2.572666109711211, 2.650477100397891, 2.852458764735376, 3.211343318481335, 3.0014482228613537, 2.45563271860859, 2.616170801723942, 2.1232357635908796]
average rmse value  : 2.6443294707089353
history of r2 square : [0.8331703982825432, 0.8626363328100879, 0.86446319797007, 0.8244229360923417, 0.8857493903002106, 0.8401215139763103, 0.6937336732131181, 0.8754718498553853, 0.81873762690018, 0.8720302458187494]
averare r2 square : 0.8370537165218996


In [20]:
import optuna
from sklearn import datasets

def objective(trial,X=X, y=y):
    
        
    n_estimators = trial.suggest_int("n_estimators", 50, 500, 50)
        
    max_features = trial.suggest_categorical("max_features",["auto","sqrt","log2"])
        
    max_depth = trial.suggest_categorical("max_depth",[None,10,20,30,40,50,60,70,80,90,100])
        
    min_samples_split = trial.suggest_int("min_samples_split",2,10,2)
        
    min_samples_leaf = trial.suggest_int("min_samples_leaf",1,5,1)
        
    bootstrap = trial.suggest_categorical("bootstrap",[True,False])

    accuracies = []
    
    kf = KFold(n_splits = 10, shuffle = True, random_state = 42)

    loss = []

    for train_index, test_index in kf.split(X):

        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        model = RandomForestRegressor(random_state=42,
                                      n_estimators=n_estimators,
                                      max_features = max_features,
                                      max_depth = max_depth,
                                      min_samples_split = min_samples_split,
                                      min_samples_leaf = min_samples_leaf,
                                      bootstrap = bootstrap) 
        model.fit(X_train, y_train) 

        y_pred = model.predict(X_test) 
        
        rmse = mean_squared_error(y_test,y_pred,squared = False)
        
        loss.append(rmse)
        average_loss = np.mean(loss)
        
    return average_loss
    
    
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials = 160)

print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)
print("best rmse is :",study.best_trial.values)

[I 2022-11-24 23:21:09,265] A new study created in memory with name: no-name-5da864f1-ad25-4bcf-ac07-4e11b67b8b1d
[I 2022-11-24 23:21:57,626] Trial 0 finished with value: 3.4370880228240024 and parameters: {'n_estimators': 250, 'max_features': 'auto', 'max_depth': 20, 'min_samples_split': 8, 'min_samples_leaf': 4, 'bootstrap': False}. Best is trial 0 with value: 3.4370880228240024.
[I 2022-11-24 23:22:48,393] Trial 1 finished with value: 2.731448417363521 and parameters: {'n_estimators': 400, 'max_features': 'auto', 'max_depth': None, 'min_samples_split': 10, 'min_samples_leaf': 1, 'bootstrap': True}. Best is trial 1 with value: 2.731448417363521.
[I 2022-11-24 23:22:56,199] Trial 2 finished with value: 3.1682225648788065 and parameters: {'n_estimators': 350, 'max_features': 'log2', 'max_depth': 70, 'min_samples_split': 8, 'min_samples_leaf': 4, 'bootstrap': True}. Best is trial 1 with value: 2.731448417363521.
[I 2022-11-24 23:23:07,027] Trial 3 finished with value: 2.7125810588090515

[I 2022-11-24 23:30:25,775] Trial 29 finished with value: 3.084367790197878 and parameters: {'n_estimators': 250, 'max_features': 'log2', 'max_depth': 20, 'min_samples_split': 2, 'min_samples_leaf': 5, 'bootstrap': False}. Best is trial 23 with value: 2.513606712784435.
[I 2022-11-24 23:30:29,681] Trial 30 finished with value: 2.8327739573455673 and parameters: {'n_estimators': 150, 'max_features': 'log2', 'max_depth': 10, 'min_samples_split': 2, 'min_samples_leaf': 1, 'bootstrap': False}. Best is trial 23 with value: 2.513606712784435.
[I 2022-11-24 23:30:33,800] Trial 31 finished with value: 2.5461847476301367 and parameters: {'n_estimators': 100, 'max_features': 'log2', 'max_depth': 60, 'min_samples_split': 4, 'min_samples_leaf': 1, 'bootstrap': False}. Best is trial 23 with value: 2.513606712784435.
[I 2022-11-24 23:30:38,504] Trial 32 finished with value: 2.5338682079375694 and parameters: {'n_estimators': 100, 'max_features': 'log2', 'max_depth': 60, 'min_samples_split': 2, 'min_

[I 2022-11-24 23:33:55,242] Trial 58 finished with value: 2.519693316089665 and parameters: {'n_estimators': 200, 'max_features': 'sqrt', 'max_depth': 30, 'min_samples_split': 2, 'min_samples_leaf': 1, 'bootstrap': False}. Best is trial 49 with value: 2.508361123376585.
[I 2022-11-24 23:34:04,368] Trial 59 finished with value: 2.794323649946383 and parameters: {'n_estimators': 300, 'max_features': 'sqrt', 'max_depth': 20, 'min_samples_split': 8, 'min_samples_leaf': 1, 'bootstrap': True}. Best is trial 49 with value: 2.508361123376585.
[I 2022-11-24 23:34:10,622] Trial 60 finished with value: 2.6550562060961664 and parameters: {'n_estimators': 150, 'max_features': 'sqrt', 'max_depth': 20, 'min_samples_split': 4, 'min_samples_leaf': 2, 'bootstrap': False}. Best is trial 49 with value: 2.508361123376585.
[I 2022-11-24 23:34:18,601] Trial 61 finished with value: 2.5103870653770075 and parameters: {'n_estimators': 150, 'max_features': 'sqrt', 'max_depth': 100, 'min_samples_split': 2, 'min_s

[I 2022-11-24 23:42:47,389] Trial 87 finished with value: 2.497061334725098 and parameters: {'n_estimators': 450, 'max_features': 'sqrt', 'max_depth': 90, 'min_samples_split': 2, 'min_samples_leaf': 1, 'bootstrap': False}. Best is trial 74 with value: 2.4951040904647206.
[I 2022-11-24 23:43:11,675] Trial 88 finished with value: 2.497061334725098 and parameters: {'n_estimators': 450, 'max_features': 'sqrt', 'max_depth': 90, 'min_samples_split': 2, 'min_samples_leaf': 1, 'bootstrap': False}. Best is trial 74 with value: 2.4951040904647206.
[I 2022-11-24 23:43:35,925] Trial 89 finished with value: 2.497061334725098 and parameters: {'n_estimators': 450, 'max_features': 'sqrt', 'max_depth': 90, 'min_samples_split': 2, 'min_samples_leaf': 1, 'bootstrap': False}. Best is trial 74 with value: 2.4951040904647206.
[I 2022-11-24 23:43:54,297] Trial 90 finished with value: 2.64987142135245 and parameters: {'n_estimators': 450, 'max_features': 'sqrt', 'max_depth': 90, 'min_samples_split': 10, 'min_

[I 2022-11-24 23:55:44,518] Trial 116 finished with value: 2.500099989142106 and parameters: {'n_estimators': 400, 'max_features': 'sqrt', 'max_depth': None, 'min_samples_split': 2, 'min_samples_leaf': 1, 'bootstrap': False}. Best is trial 74 with value: 2.4951040904647206.
[I 2022-11-24 23:57:44,524] Trial 117 finished with value: 3.419986364485357 and parameters: {'n_estimators': 450, 'max_features': 'auto', 'max_depth': 90, 'min_samples_split': 2, 'min_samples_leaf': 1, 'bootstrap': False}. Best is trial 74 with value: 2.4951040904647206.
[I 2022-11-24 23:58:00,036] Trial 118 finished with value: 2.7459692458259113 and parameters: {'n_estimators': 500, 'max_features': 'sqrt', 'max_depth': 10, 'min_samples_split': 2, 'min_samples_leaf': 1, 'bootstrap': False}. Best is trial 74 with value: 2.4951040904647206.
[I 2022-11-24 23:58:24,085] Trial 119 finished with value: 2.497061334725098 and parameters: {'n_estimators': 450, 'max_features': 'sqrt', 'max_depth': 80, 'min_samples_split': 2

[I 2022-11-25 00:09:25,201] Trial 145 finished with value: 2.4951040904647206 and parameters: {'n_estimators': 400, 'max_features': 'sqrt', 'max_depth': 20, 'min_samples_split': 2, 'min_samples_leaf': 1, 'bootstrap': False}. Best is trial 74 with value: 2.4951040904647206.
[I 2022-11-25 00:09:40,054] Trial 146 finished with value: 2.6543901380866832 and parameters: {'n_estimators': 400, 'max_features': 'sqrt', 'max_depth': 20, 'min_samples_split': 2, 'min_samples_leaf': 1, 'bootstrap': True}. Best is trial 74 with value: 2.4951040904647206.
[I 2022-11-25 00:09:59,896] Trial 147 finished with value: 2.4951040904647206 and parameters: {'n_estimators': 400, 'max_features': 'sqrt', 'max_depth': 20, 'min_samples_split': 2, 'min_samples_leaf': 1, 'bootstrap': False}. Best is trial 74 with value: 2.4951040904647206.
[I 2022-11-25 00:10:17,257] Trial 148 finished with value: 2.4970943491488624 and parameters: {'n_estimators': 350, 'max_features': 'sqrt', 'max_depth': 20, 'min_samples_split': 2

Number of finished trials: 160
Best trial: {'n_estimators': 400, 'max_features': 'sqrt', 'max_depth': 20, 'min_samples_split': 2, 'min_samples_leaf': 1, 'bootstrap': False}
best rmse is : [2.4951040904647206]


In [21]:
params = study.best_trial.params

In [22]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate

kf = KFold(n_splits = 10, shuffle = True, random_state = 42)

loss_history = []
accuracy = []

for train_index, test_index in kf.split(X):

    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    model = RandomForestRegressor(random_state=42,**params) 
    model.fit(X_train, y_train) 

    y_pred = model.predict(X_test) 
    loss_history.append(mean_squared_error(y_test, y_pred,squared=False))
    accuracy.append(r2_score(y_test,y_pred,))
print("history of rmse :", loss_history)
print("average rmse value  :", np.mean(loss_history))
print("history of r2 square :",accuracy)
print("averare r2 square :" ,np.mean(accuracy))

history of rmse : [2.751592351131444, 2.057569019693505, 2.6241962999802424, 2.4408140111261645, 2.790219982584365, 3.0399828833037623, 2.662487183796346, 2.150552291257922, 2.40395386441822, 2.0296730173552366]
average rmse value  : 2.4951040904647206
history of r2 square : [0.8393439552940297, 0.8748809350031885, 0.8589792497168733, 0.8511019381241777, 0.8906807443937016, 0.8567288231043195, 0.7590024708453365, 0.9044917850299496, 0.846951974831685, 0.88306001049335]
averare r2 square : 0.8565221886836613


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2, random_state = 42)
model = RandomForestRegressor(random_state = 42, **params)
model.fit(X_train,y_train)

y_pred = model.predict(X_test)
print(r2_score(y_test,y_pred))
print(mean_squared_error(y_test,y_pred,squared=False))